In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/binary-buzz-cyber-labs-ml-25/train.csv
/kaggle/input/binary-buzz-cyber-labs-ml-25/test.csv


In [2]:
import pandas as pd

train_df = pd.read_csv('/kaggle/input/binary-buzz-cyber-labs-ml-25/train.csv')
test_df = pd.read_csv('/kaggle/input/binary-buzz-cyber-labs-ml-25/test.csv')

train_df.head()





,id,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x3064,x3065,x3066,x3067,x3068,x3069,x3070,x3071,x3072,target
0,3344,255,255,255,255,255,255,255,255,255,...,254,254,255,255,255,255,255,255,255,0
1,317,79,43,15,82,42,15,82,42,14,...,73,44,31,50,39,28,56,30,15,0
2,2288,191,201,211,192,202,211,193,202,211,...,170,171,175,169,170,173,169,170,173,0
3,1456,255,255,253,255,255,253,253,252,250,...,88,109,158,43,76,144,3,26,93,0
4,2205,99,95,40,106,98,39,101,93,29,...,160,114,57,183,134,76,226,182,111,0


In [3]:
X = train_df.drop(['target'], axis=1)
y = train_df['target'].values

X_test = test_df.copy()

In [4]:
X = X.values
X_test = X_test.values

mean = X.mean(axis=0)
std = X.std(axis=0)

X = (X - mean) / std
X_test = (X_test - mean) / std
X = np.concatenate([X, X**2], axis=1)
X_test = np.concatenate([X_test, X_test**2], axis=1)


In [5]:

split = int(0.8 * len(X))

X_train, X_val = X[:split], X[split:]
y_train, y_val = y[:split], y[split:]


In [6]:
class LogisticRegressionScratch:
    def __init__(self, lr=0.01, epochs=1000):
        self.lr = lr
        self.epochs = epochs

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def fit(self, X, y):
        self.m, self.n = X.shape
        self.W = np.random.randn(self.n) * 0.01
        self.b = 0
        self.losses = []

        for _ in range(self.epochs):
            z = np.dot(X, self.W) + self.b
            y_pred = self.sigmoid(z)

            loss = -(1/self.m) * np.sum(
                y*np.log(y_pred+1e-9) + (1-y)*np.log(1-y_pred+1e-9)
            )
            self.losses.append(loss)

            dw = (1/self.m) * np.dot(X.T, (y_pred - y))+0.01*self.W
            db = (1/self.m) * np.sum(y_pred - y)

            self.W -= self.lr * dw
            self.b -= self.lr * db

    def predict(self, X):
        probs = self.sigmoid(np.dot(X, self.W) + self.b)
        return (probs >= 0.5).astype(int)


In [7]:
model = LogisticRegressionScratch(lr=0.05, epochs=6000)
model.fit(X_train, y_train)


In [8]:
y_pred = model.predict(X_val)
accuracy = np.mean(y_pred == y_val)

accuracy


0.9025

In [9]:
def he_init(n_in, n_out):
    return np.random.randn(n_in, n_out) * np.sqrt(2 / n_in)


In [10]:
class NeuralNetworkScratch:
    def __init__(self, input_dim, hidden_dim=32, lr=0.045, epochs=5000):
        self.lr = lr
        self.epochs = epochs

        self.W1 = he_init(input_dim, hidden_dim)
        self.b1 = np.zeros((1, hidden_dim))

        self.W2 = np.random.randn(hidden_dim, 1) * 0.01
        self.b2 = np.zeros((1, 1))

        self.losses = []

    def relu(self, z):
        return np.maximum(0, z)

    def relu_deriv(self, z):
        return (z > 0).astype(float)

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def forward(self, X):
        self.Z1 = np.dot(X, self.W1) + self.b1
        self.A1 = self.relu(self.Z1)

        self.Z2 = np.dot(self.A1, self.W2) + self.b2
        self.A2 = self.sigmoid(self.Z2)

        return self.A2

    def compute_loss(self, y, y_pred):
        m = len(y)
        return -(1/m) * np.sum(
            y*np.log(y_pred + 1e-9) + (1-y)*np.log(1 - y_pred + 1e-9)
        )

    def backward(self, X, y):
        m = len(y)

        dZ2 = self.A2 - y.reshape(-1, 1)
        dW2 = (1/m) * np.dot(self.A1.T, dZ2)
        db2 = (1/m) * np.sum(dZ2, axis=0, keepdims=True)

        dA1 = np.dot(dZ2, self.W2.T)
        dZ1 = dA1 * self.relu_deriv(self.Z1)
        dW1 = (1/m) * np.dot(X.T, dZ1)
        db1 = (1/m) * np.sum(dZ1, axis=0, keepdims=True)

        self.W2 -= self.lr * dW2
        self.b2 -= self.lr * db2
        self.W1 -= self.lr * dW1
        self.b1 -= self.lr * db1

    def fit(self, X, y):
        for _ in range(self.epochs):
            y_pred = self.forward(X)
            loss = self.compute_loss(y, y_pred)
            self.losses.append(loss)
            self.backward(X, y)

    def predict(self, X):
        probs = self.forward(X)
        return (probs >= 0.5).astype(int).flatten()


In [11]:
nn = NeuralNetworkScratch(
    input_dim=X_train.shape[1],
    hidden_dim=64,
    lr=0.003,
    epochs=8000
)

nn.fit(X_train, y_train)


In [12]:
y_pred = nn.predict(X_val)
accuracy = np.mean(y_pred == y_val)
accuracy


0.9075

In [13]:
final_nn = NeuralNetworkScratch(
    input_dim=X.shape[1],
    hidden_dim=64,
    lr=0.003,
    epochs=8000
)

final_nn.fit(X, y)
test_preds = final_nn.predict(X_test)


In [14]:
submission = pd.DataFrame({
    "id": test_df["id"],
    "target": test_preds
})
submission.to_csv("submission3.csv", index=False)
